<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Nucleo F303RE Zero Hardware Board

# Board 02 - SLab Setup

This document describes the setup of the hardware board that will be the basic instrument in the SLab Python project.  
Next the SLab environment is described

Version 1.3 (12/4/2019)
License information is at the end of the document

---
**Bill Of Materials (BOM):**

1x red LED  
1x $470\Omega$ resistor

---


## Introduction

At the end of the previous document we had a working SLab Python environment, we programmed the hardware board with a SLab firmware and we checked that it can communicate with the PC.

Typical microcontroller boards usually have poor ADC and DAC converters performance under non ideal conditions. This is due, as we will explain, to loading effects. To improve the board performace, the first **SLab** implementation, that we will call **classic**, used external componets to buffer the converters. The **SLab** implementation described in this document is called **Zero** because it doesn't require any external buffer component. 

This document is specific to the use of the **STM32 Nucleo F303RE** board as the SLab hardware board. Keep that in mind if you use other different board for your SLab setup.

## Need for a buffer setup

Microcontroller (MCU) devices usually include a CPU, flash memory, RAM memory and several peripherals in a single integrated circuit. Most MCU input/outputs work with very low current levels because current drivers take-up too much silicon space. That means that ADCs and DACs are quite sensitive to loading effects. If you are not convinced about that, the rest of this section gives a more detailed explanation. If you don’t want to mess with the details, you can skip to the next section.

Loading effects appear when a load takes current from a source. Take as an example the following circuit where a DAC that we use to generate a voltage $V_{DAC}$, with an output resistance $R_{DAC}$, is connected to a resistive load $R_{Load}$.

![fig01](images/Board_02/fig01.png)

The voltage at the load will be:

$$V_{Load} = V_{DAC} \frac{R_{Load}}{R_{Load}+R_{DAC}}$$

The voltage at the load won’t be the same as the value we programmed on the DAC. Let’s calculate the how high the load resistance needs to be so that the error is less than 1%:

$$\frac{V_{Load}}{V_{DAC}} \geq 0.99 \qquad R_{Load} \geq 99 \cdot R_{DAC}$$

The **F303RE** board features a DAC with a maximum specified output resistance of $15k\Omega$ as it can be seen on the MCU datasheet:

![fig01](images/Board_02/DAC_Specs.png)

That means that, in order to have a less than 1% error, the load must be over $1,5M\Omega$. Moreover, as in this board the DAC has 12 bit resolution, we need to have an error of less than about 0,01% for it to be negligible. That will increase the load requirements to over $150M\Omega$. The end result is that, if our budget error is small, we cannot connect the DAC to any load at all.

The solution to the problem is to add a buffer with unity gain on the output of the DAC that does not load it:

![fig02](images/Board_02/fig02.png)

We have a similar problem with the ADCs. The MCU ADCs are used to read the voltage on a circuit node. We can model any DC circuit with a Thevenin equivalent. The model of the ADC is more complex that the one of the DAC, however. Typical [SAR ADCs](https://en.wikipedia.org/wiki/Successive_approximation_ADC) can be modeled as a RC network and a switch. Whenever you want to read an ADC input, you close the switch so that the $V_{ADC}$ equals the $V_{TH}$ voltage we want to measure.

![fig03](images/Board_02/fig03.png)

It takes time to load the $C_{ADC}$ capacitance to a voltage close enough to the open circuit $V_{TH}$ voltage. In the case of the F303RE board, the manufacturer recommends $R_{TH}$ to be always below $47k\Omega$. At maximum sample rates, however, it shall be below $18\Omega$ if we want the full 12 bit resolution of the ADC.

The $C_{ADC}$ capacitor is depleted on each reading so we need to drain some charge from the circuit every time we read it with the ADC.
 
In general, although at low sample frequencies the problem is not as big as in the DACs, we also need to drive the ADC inputs with a unity gain buffer:

![fig04](images/Board_02/fig04.png)

## SLab Zero DAC buffering

In the **SLab Classic** solution we used six external **operational amplifiers** to buffer, with unity gain, the 2 DACs and 4 ADCs of the system.

![fig05](images/Board_02/fig05.png)

In the **SLab Zero** solution we use the resources already included inside the MCU to provide the needed buffering.

The F303 board MCU includes one buffer for each DAC. If you recall the DAC table previously shown in this document table, the $15k\Omega$ output resistance is for the case that doesn't use the internal buffer. When using the internal buffer, the table indicates that you can drive resistances as low as   $5k\Omega$ if connected to ground or as low as $25k\Omega$ if connected to Vdd.

One of the **Notebooks** inside the **Demo** folder, called **"DAC Analysis"** gives a better insight on the DAC buffering.

In practice we can use smaller resistor loads if we relax the measurement accuracy.

All in all the solution for the **SLab Zero** implementation is to rely on the internal DAC buffers. That will suit low load circuits. For the cases where more current is needed we will use two alternative options:

* Use an ADC connected to the DAC output. That way, although the DAC output can be different from the programmed value, we will measure the **real** applied voltage.


* Use of an external **operational amplifier**, like the **MCP6002** to provide a second buffering layer. This buffer will be part of the test circuit when it is needed.

There is one additional gotcha associated to the **F303 Board**. The second DAC is hardware connected to the **user LED** of the board. That induces some loading effect on the DAC. That means that the load capabilities of **DAC2** are poorer than the ones on **DAC1**.

## SLab Zero ADC buffering

The **Classic SLab** solution used 4 external operational amplifiers (opamps) to buffer the circuit measurement nodes before reaching the MCU ADC.

The F303 board MCU includes, as part of its internal peripheral set, four opamps. That means that we can use them instead of using external devices. Unfortunatelly one of the internal opamps output nodes conflicts with the pin used for the USB serial communication. That means that one opamp is out if we don't want to modify the board hardware.

That leaves us with three internal opamps. Those opamps have input multiplexers that enables us to connect them to up to six free external pins. In the **F303 Zero** solution, we use 8 ADC lines, 6 of them buffered and 2 unbuffered.

In AC operation we want to operate the ADCs as fast as possible. The F303 board MCU features four independent hardware ADCs, that means that we can measure four voltages at the same time without needing to multiplex the signals. That's why, although the **F303 Zero** solution features 8 ADC lines, we only use 4 of them (one of them unbuffered) in most AC measurements.

Channels 5 to 8 in the **F303 Zero** can be read on AC, but only one at a time using the **singleWaveResponse** command.

## SLab F303 Zero Pinout

As explained, **SLab F303 Zero** solution just uses the **STM32 Nucleo F303RE** demonstration board as the SLab **hardware board** without adding any component. The following figure shows an block diagram for this board.

![SLab_Zero](images/Board_02/SLab_Zero.png)

It includes a total of 26 pins:

Three power pins:

* A 3.3V supply
* A Ground connection
* A 5V supply (Seldom used on SLab)

Two buffered DACs:

* DAC1 
* DAC2

Eight ADCs. Four of them available on DC and AC measurements:

* ADC1
* ADC2
* ADC3
* ADC4 (Unbufferred)

Four of them available for DC measurements and in single wave response for AC measurements:

* ADC5
* ADC6
* ADC7
* ADC8 (Unbufferred)

Thirteen digital I/O lines:

* DIO0 ... DIO12

In the **SLab Classic** solution we only used pins available on the female Arduino headers of the **F303 Board**. This is too restrictive, in the **SLab Zero** solution we access the pins using the **Morpho** male headers, shown in the red boxes in the figure below.

![Morpho](images/Board_02/Morpho.jpg)

Observe that, in the Nucleo board, the **Arduino** headers have their pins labeled whereas the **Morpho** ones don't. That, added to the fact that there are much more **Morpho** pins than **Arduino** ones, makes difficult to locate the proper pin to connect.

The following figure shows the pinout used on the **F303 Zero** solution. The image only shows the **Morpho** connectors **CN7** and **CN10** because all **Arduino** pins are also available as **Morpho** pins. 

![F303RE_Zero_Pinout](images/Board_02/F303RE_Pinout.png)

The left block shows the connections on the left **Morpho** header (CN7), whereas the righ block does the same for the right **Morpho** header (CN10). As you can see there are a total of 76 **Morpho** pins (Two headers with two columns of 19 rows).

Note that, in our experiments, the $V_{DD}$ rail will be connected to the $3V3$ pin, we won't use the $5V$ pin.

In most **SLab experiments** we will only use eight pins:

* $3V3$ and $GND$ supplies
* **DAC1** and **DAC2** voltage generators
* **ADC1**, **ADC2**, **ADC3** and **ADC4** voltage measurements

## Breadboard Setup

As we remember, from previous documents, we will use a solderless **breadboard** to build our circuits. 

Our solderless [breadboard](https://en.wikipedia.org/wiki/Breadboard) will be something like that one:

![fig08](images/Board_02/fig08.png)

The board includes multiple connecting points. Some connecting points are connected together as shown in the following figure. In particular there are two red lines of connections, tipically for Vdd, and two blue lines of connections, typically for ground.

![fig09](images/Board_02/fig09.png)

The first thing that we will implement in the **breadboard** is a LED circuit. We will also connect together the two red vdd power strips of the board and the two blue ground strips.

![LED_Circuit](images/Board_02/LED_Circuit.png)

The following figure shows an implementation of this circuit. In the case of the red LED, the long terminal is the one that needs to be connected to the positive rail. If you connect it wrong, it just won’t lit.

![BB_LED](images/Board_02/BB_LED.jpg)

In the case of the red LED, the long terminal is the one that needs to be connected to the positive rail. If you connect it wrong, it just won’t lit.

Next step is to connect the breadboard to the F303RE hardware board. The following figure shows the F303RE board with two board LEDs, **LD1** and **LD3** marked using yellow boxes.

![Nucleo_LEDs](images/Board_02/Nucleo_LEDs.jpg)

Connect the board to a PC. Led LD1 at the top of the board should light on red or green. Led LD3 in the middle of the board should light in red.

We will use **Female** to **Male** [jump wires](https://en.wikipedia.org/wiki/Jump_wire) to connect the **SLab hardware board** to the **breadboard**. Note that most **SLab** pins on the previous shown table have specific colors. We will try to use **jump wires** with the same colors:

* <font color=red>Red</font> for 3.3V Vdd
* Black for Ground

![Jump Power](images/Board_02/Jump_Power.jpg)

* <font color=yellow>Yellow</font> for DAC1
* <font color=orange>Orange</font> for DAC2

![Jump DACs](images/Board_02/Jump_DACs.jpg)

* <font color=blue>Blue</font> for ADC1 and ADC5
* <font color=green>Green</font> for ADC2 and ADC6
* <font color=lilac>Lilac</font> for ADC3 and ADC7
* <font color=brown>Brown</font> for the unbuffered ADC4 and ADC8 ADCs

![Jump ADC 1-4](images/Board_02/Jump_ADC1_4.jpg)

Most SLab experiments require a maximum of 4 ADCs, when we need to use ADCs 4 to 8, we can mark the male end of the wire, with an orange mask in this case, to make them different from the other ADCs:

![Jump ADC 5-8](images/Board_02/Jump_ADC5_8.jpg)

In SLab we can also use, sometimes, the digital I/O lines the color coding for these lines will be:

* White for DIO0 and DIO8
* <font color=grey>Grey</font> for DIO1 and DIO9
* <font color=yellow>Yellow</font> for DIO2 and DIO10
* <font color=orange>Orange</font> for DIO3 and DIO11

![Jump DIO 0-3](images/Board_02/Jump_DIO0_3.jpg)

* <font color=blue>Blue</font> for DIO4 and DIO12
* <font color=green>Green</font> for DIO5 and DIO13
* <font color=lilac>Lilac</font> for DIO6 and DIO14
* <font color=brown>Brown</font> for DIO7 and DIO15

![Jump DIO 4-7](images/Board_02/Jump_DIO4_7.jpg)

Note that no currently available board features 16 DIO lines. The F303 board only features 13 I/O lines from DIO0 to DIO12.

In order to separate the **digital lines** from the **analog** ones, you can use a white mark at the male end of the wires like is shown in the above figures.

To ease the transpor of the SLab system, you can join together the **F303 Board** and the **breadboard** like in the **Long Board** shown below:

![Long_Board](images/Board_02/Long_Board.jpg)

## First test of the setup

The first circuit we will implement is shown in the figure below. We will connect the **LED-Resistor** series combination to **Vdd** and **GND**, and we will connect **DAC1** to **ADC1**. If you have only **Female to Male** jump wires you need to do that last connection on the **breadboard**. If you have **Female to Female** jump wires, you can perform a direct connection.

![First_Circuit](images/Board_02/First_Circuit.png)

In the schematics we will show the **DACs** and **Vdd** as voltage sources. **Red** for Vdd, **Yellow** for **DAC1** and **Orange** for **DAC2**. The **GND** wires will be marked with a **ground** symbol and a **Black** box. Sometimes, however, the power connections are implicit to the circuit and are not shown.  
The **ADCs** will be marked with their particular color, blue in the case of **ADC1** on this circuit.  
Although we will leave always the **Red LED** on the breadboard, we won't usually show it on the schematics as it is not usually related to the circuits we will test.

In **SLab** we won't, except for this first circuit, draw the circuits that way. We will use an equivalent schematic like in the figure below.

![First_Circuit_2](images/Board_02/First_Circuit_2.png)

Observe that there are four **colored boxes** on the circuit above. One for each used pin on the **hardware board**. For each pin we will use a **Jump Wire** with the same color. 

The following figure shows the **Long Board** with the proposed wired connections.

![Connections](images/Board_02/Connections.jpg)

As soon as you connect both power wires to the breadboard, the **Red LED** should light up. If it does not, it is probably backwards. If you see the **F303 Board** LED **LD3** turn off or the board disconnects from the computer, probably you are making a **short circuit**. Desconnect the USB board, then all the wires and repeat again.

We can now test this simple circuit. If you are reading this document from **Jupyter** in a properly configured **SLab** system, you can execute the **code cell** below to run the code that tests the circuit.

In [ ]:
# Import SLab
import slab

# Connect to the board
slab.connect()

# Perform a DC sweep
slab.dcSweepPlot(1,0,3,0.1,1)

# Disconnect from the board
slab.disconnect()

If the connection is ok, you should see above a graph with a blue line with somewhat equal X and Y values. As the **DAC** is directly connected to the **ADC** both voltages should match.

## Wire Docking

We use colored **jump wires** to ease knowing which wire goes to which **SLab** pin. To connect the wires to the board, however, you need to follow the board diagram. To reduce the chance of errors, you can leave the wires permanently connected to the board when not in use, whenever it is possible.

The following figure shows a **SLab Fat Board** with the cables for **Vdd**, **GND**, the two **DACs** and four of the eight **ADCs**.

![Long Board](images/Board_02/Long_Board2.jpg)

You can see that the two power **Vdd** and **GND** wires, the two **DAC1** and **DAC2** wires and the four **ADC1** to **ADC4** wires are docked on the breadboard. The following figures shows that in detail:

![Cable Docking](images/Board_02/docking.jpg)

We can see that the power wires are connected to the **power rails**, **DAC1** wires is connected to the same node as **ADC1**, **ADC3** and **ADC4** and that **DAC2** is connected to the same node thatn **ADC2**. We will call this the **"SLab Standard Docking"**.

You don't need to follow the Standard Docking. **ADC** wires, as they relate to inputs can be connected to any floating node, to **Vdd**, to **GND** or to any **DAC** output.  
**DAC** wires, as they belong to outputs are more restricted. They shall never be connected to **Vdd**, **GND** or to any other **DAC**.

The main advantage of the **SLab Standard Docking** is that its connections are the ones needed to check the board calibration. So, if you leave the board in this state when not in use, you can do the calibration check at the start of the experiments without needing to touch any wire.

When you are performing measurements, for instance, if you need **DAC1** and **ADC1** and not **DAC2** or the other **ADCs**, you can leave the other wires as they are in the docking position.

Note that the **SLab Standard Docking** only includes fou ADC wires. As you know, the **F303 Board** SLab solution features four more ADCs, from **ADC5** to **ADC8** and 13 digital I/O lines from **DIO0** to **DIO12** that can also be used. 

That ends this **SLab Setup** document. Next we will talk about the **calibration** of the board to obtain better measurements.

## Easing board usage

This is the last requied document to use the **STM32F303RE Nucleo64** board on **SLab** the rest of documents in this section are optional. 

The [DAC Improvement](http://localhost:8888/notebooks/Boards/Nucleo_F303RE_Zero/Board%2003%20-%20DAC2%20Improvement.ipynb) document explains hoy to have have a better response on **DAC2** by disconnecting the **User LED** from it.

The [SLab Card](http://localhost:8888/notebooks/Boards/Nucleo_F303RE_Zero/Board%2004%20-%20Board%20Pin%20Locator.ipynb) document explains how to put a card over the board so that you know which signal goes to which board pin.

![SLab card](images/Board_02/slab_card_onboard.jpg)

## References

**SLab Python References**  
Those are the reference documents for the SLab Python modules. They describe the commands that can be carried out after importing each module.  
They should be available in the SLab/Doc folder.

**TinyCad**  
Circuit images on this document have been drawn using the free software TinyCad  
https://sourceforge.net/projects/tinycad/

**Fritzing**  
The Fritzing program has been used to draw the breadboard images.  
http://fritzing.org/home/

## Document license

Copyright  ©  Vicente Jiménez (2018-2019)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">